In [13]:
import sys
sys.path.append('core')
from PIL import Image
import os
import argparse

import os
import cv2
import glob
import numpy as np
import torch
from PIL import Image

from raft import RAFT
from utils import flow_viz
from utils.utils import InputPadder
from random import sample
from matplotlib import pyplot as plt
import pandas as pd
import time

In [14]:
def load_image(imfile):
    im = Image.open(imfile)
    img = np.array(im).astype(np.uint8)
    img = torch.from_numpy(img).permute(2, 0, 1).float()
    return img[None].to(DEVICE)

def process_image(img):
    img = torch.from_numpy(img).permute(2, 0, 1).float()
    return img[None].to(DEVICE)

def viz(img, img2, flo):
    img = img[0].permute(1,2,0).cpu().numpy()
    img2 = img2[0].permute(1,2,0).cpu().numpy()
    flo = flo[0].permute(1,2,0).cpu().numpy()
    plt.figure(figsize=(14,7))
    plt.imshow(np.concatenate([img.astype(np.uint8),img2.astype(np.uint8)],axis=0),)
    plt.figure()
    plt.imshow(np.abs(img-img2))
    # map flow to rgb image
    flo_viz = flow_viz.flow_to_image(flo)
    img_flo = np.concatenate([img.astype(np.uint8), flo_viz], axis=0)
    plt.figure()
    plt.imshow(flo_viz)
    plt.figure()
    plt.hist(flo.flatten(), bins=np.arange(-25,25))
    plt.show()
    
def process_flow(flow,max_flow=20):
    flow = flow[0].permute(1,2,0).cpu().numpy()
    flow[flow>max_flow] = max_flow
    flow[flow<(-1*max_flow)] = -1 * max_flow
    
    flow_image = 255 * (flow + max_flow) / (2*max_flow)
    flow_image = flow_image.astype(np.uint8)
    flow_image = np.concatenate([flow_image, np.zeros((flow.shape[0],flow.shape[1],1),dtype=np.uint8)], axis=2)
    return flow_image

def load_video(video_fname):
    frames = []
    cap = cv2.VideoCapture(video_fname)
    ret = True
    fps = None
    fourcc = None
    while ret:
        ret, img = cap.read() # read one frame from the 'capture' object; img is (H, W, C)
        if ret:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            frames.append(img)
            fps = cap.get(cv2.CAP_PROP_FPS)
            fourcc = cap.get(cv2.CAP_PROP_FOURCC)
    return frames, fps, fourcc

def write_frame_list_to_video(frames_list, fps, video_fname):
    h, w, _ = frames_list[0].shape
    num_frames = len(frames_list)
    fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    writer = cv2.VideoWriter(video_fname, fourcc, fps, (w, h))
    for frame in frames_list:
        frame_fixed = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        writer.write(frame_fixed)
    writer.release()

In [17]:
parser = argparse.ArgumentParser()
parser.add_argument('--model', help="restore checkpoint")
parser.add_argument('--path', help="dataset for evaluation")
parser.add_argument('--small', action='store_true', help='use small model')
parser.add_argument('--mixed_precision', action='store_true', help='use mixed precision')
parser.add_argument('--alternate_corr', action='store_true', help='use efficent correlation implementation')
args = parser.parse_args("--model=models/raft-sintel.pth --path=demo-frames".split())

In [18]:
DEVICE='cuda'
model = torch.nn.DataParallel(RAFT(args))
model.load_state_dict(torch.load(args.model))
model = model.module
model.to(DEVICE)
model.eval()

RAFT(
  (fnet): BasicEncoder(
    (norm1): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (relu1): ReLU(inplace=True)
    (layer1): Sequential(
      (0): ResidualBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (relu): ReLU(inplace=True)
        (norm1): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
        (norm2): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      )
      (1): ResidualBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (relu): ReLU(inplace=True)
        (norm1): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=Fa

In [19]:
input_dir = 'official'
all_videos = glob.glob(input_dir +'/**/**/*.mp4')
all_videos = sorted(sample(all_videos,3))
output_dir = 'official_flow'
#rgb_frames = sorted(os.listdir(os.path.join(rgb_dir,'rgb')))

In [20]:
with torch.no_grad():
    df = pd.DataFrame(([True] * len(all_videos)), index =all_videos,
                                              columns =['has_flow'])
    for video_fname in all_videos:
        print(video_fname)
        rgb_frames, fps, fourcc = load_video(video_fname)
        if len(rgb_frames) < 2:
            df.loc[video_fname,'has_flow']=False
        flow_video = []
        start=time.time()
        for image1, image2 in zip(rgb_frames[:-1], rgb_frames[1:]):
#             k+=1
#             print(k)
            image1_proc = process_image(image1)
            image2_proc = process_image(image2)
            padder = InputPadder(image1_proc.shape)
            image1_pad, image2_pad = padder.pad(image1_proc,image2_proc)
            #start=time.time()
            flow_low_pad, flow_up_pad = model(image1_pad, image2_pad, iters=10, test_mode=True)
            #end = time.time()
            flow_up = padder.unpad(flow_up_pad)
            #viz(image1, image2, flow_up)
            flow_image = process_flow(flow_up)
            flow_video.append(flow_image)
#             if k >10:
#                 break
#             print(end-start)
        end = time.time()
        print(end-start, 'flow computation')
        start=time.time()
        save_filename = video_fname.replace(input_dir, output_dir)
        save_dir = os.path.dirname(save_filename)
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
        write_frame_list_to_video(flow_video, fps, save_filename)
        end = time.time()
        print(end-start, 'video saving')
    df.to_csv('video_has_flow.csv')


official/train_256/bobsledding/0PaakVpDsuo_000002_000012.mp4
8.525222063064575 flow computation


OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


0.32456469535827637 video saving
official/train_256/country_line_dancing/GlfV6UM-Zns_000472_000482.mp4
8.384076595306396 flow computation


OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


0.39324259757995605 video saving
official/train_256/exercising_with_an_exercise_ball/qhnRZhxuQ6c_000039_000049.mp4
7.241694450378418 flow computation
0.0808401107788086 video saving


OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


In [11]:
def process_frame_list(frames_list):
    f1 = frames_list[:-1] 
    f2 = frames_list[1:]
    f1_proc = [process_image(frame) for frame in f1]
    f2_proc = [process_image(frame) for frame in f2]
    padder = InputPadder(f1_proc[0].shape)
    f1_proc_pad = [padder.pad(frame_proc)[0] for frame_proc in f1_proc]
    f2_proc_pad = [padder.pad(frame_proc)[0] for frame_proc in f2_proc]
    return torch.cat(f1_proc_pad), torch.cat(f2_proc_pad)

with torch.no_grad():
    for video_fname in [single_video]:
        start=time.time()
        video_frame_list = load_video(video_fname)
        f1, f2 = process_frame_list(video_frame_list)
        start=time.time()
        flow_low_pad, flow_up_pad = model(f1.contiguous(), f2.contiguous(), iters=10, test_mode=True)
        end = time.time()
        print(end-start)

RuntimeError: cuDNN error: CUDNN_STATUS_NOT_SUPPORTED. This error may appear if you passed in a non-contiguous input.

In [28]:
x = ['a1', 'b2', 'c3' , 'd4', 'e5', 'f6', 'g7', 'h8','i9','j10', 'k11', 'l12', 'm13']
split_list = np.array_split(np.array(x, dtype=object), 3)

In [31]:
list(split_list[0])

['a1', 'b2', 'c3', 'd4', 'e5']